In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup, Comment
from tqdm import tqdm

In [2]:
SCORE_DIR = "E:/Projects/NBA Prediction/html"

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [4]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [5]:
def parse_html(box_score):
    with open(box_score, "r", encoding='utf-8') as f:
        html = f.read()
    
    soup = BeautifulSoup(html)
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))

    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thread")]

    return soup, comments

In [6]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score

In [7]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [8]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season